In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import statsmodels.api as sm

# 資料前處理

## 讀取資料

1. 首先，讀取2019年的資料後，將我們認為不必要的屬性刪除
2. 接著，讀取縣市的編碼後，利用pandas的merge將兩表以編號為基準合併
3. 最後，刪除合併後多餘的屬性

In [20]:
df = pd.read_csv('tw-rental-data/2019-dedup.csv')
df = df.drop(['重複物件數', '最大物件編號', '最小物件編號', '最大物件首次發現時間', '最小物件首次發現時間', '租屋平台', 
              '最後出租時間', '最大出租所費天數', '押金類型', '押金金額', '月管理費', '月停車費', '每坪租金（含管理費與停車費）', 
              '建物樓高', '距頂樓層數', '格局編碼（陽台/衛浴/房/廳）', '性別限制', '有產權登記？', '刊登者類型', '最大刊登者編碼', 
              '仲介資訊'], axis=1)
city = pd.read_csv('tw-rental-data/編碼表/編碼表.縣市.csv')
df = df.merge(city, left_on='縣市', right_on='編碼')
df = df.drop(['縣市', '編碼', '發行日期'], axis=1)
df

C:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (12,13,27) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,鄉鎮市區,約略地點範圍,房屋曾出租過,月租金,押金月數,需要管理費？,提供車位？,需要停車費？,建築類型,物件類型,...,提供家具_熱水器？,提供家具_冷氣？,提供家具_沙發？,提供家具_洗衣機？,提供家具_衣櫃？,提供家具_冰箱？,提供家具_網路？,提供家具_第四台？,提供家具_天然瓦斯？,代表
0,1806,"(23.009119, 120.221691, 23.009119, 120.221691)",1,2000,2.0,-,F,-,1,3,...,T,F,F,T,T,T,T,F,F,臺南市
1,1824,"(22.9918675, 120.2157114, 22.9918675, 120.2157...",2,13600,1.0,F,F,F,2,1,...,T,T,T,T,T,T,T,T,T,臺南市
2,1829,"(23.0197438, 120.2498191, 23.0197438, 120.2498...",1,11000,2.0,F,T,F,2,0,...,T,T,T,T,T,T,F,F,T,臺南市
3,1829,-,1,10000,2.0,T,F,-,0,0,...,T,T,F,T,F,T,F,F,T,臺南市
4,1824,"(22.987148, 120.2168226, 22.987148, 120.2168226)",1,5000,2.0,F,F,-,2,1,...,T,T,T,T,T,T,F,F,F,臺南市
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
747608,1505,"(23.5742536, 119.5784689, 23.5742536, 119.5784...",0,4300,1.0,F,F,-,1,1,...,T,T,F,T,T,T,T,F,F,澎湖縣
747609,1505,"(23.5661176, 119.56743, 23.5661176, 119.56743)",1,10000,2.0,-,F,-,1,2,...,T,T,T,T,T,T,T,T,F,澎湖縣
747610,1505,"(23.57622, 119.570249, 23.57622, 119.570249)",0,35000,-,T,F,-,2,0,...,T,T,T,T,T,T,T,T,T,澎湖縣
747611,65535,"(25.0634479, 121.5874002, 25.0634479, 121.5874...",0,-,2.0,T,T,F,2,0,...,-,-,-,-,-,-,-,-,-,不明


## 篩選區域

### 只選取北北基範圍

1. 將縣市為台北市、新北市和基隆市的index記起來後進行篩選刪除其餘縣市
2. 接著，再對此資料集之縣市取虛擬函數

In [21]:
mask1 = df['代表'] == '臺北市'
mask2 = df['代表'] == '新北市'
mask3 = df['代表'] == '基隆市'
df = df[(mask1 | mask2 | mask3)]
dummies = pd.get_dummies( df.loc[:, '代表'], prefix='代表' ) 
df = pd.concat( [df, dummies], axis = 1 )
df = df.drop(['代表'], axis=1)
df

,鄉鎮市區,約略地點範圍,房屋曾出租過,月租金,押金月數,需要管理費？,提供車位？,需要停車費？,建築類型,物件類型,...,提供家具_沙發？,提供家具_洗衣機？,提供家具_衣櫃？,提供家具_冰箱？,提供家具_網路？,提供家具_第四台？,提供家具_天然瓦斯？,代表_基隆市,代表_新北市,代表_臺北市
32463,1707,"(25.068092, 121.519283, 25.068092, 121.519283)",1,10200,2.0,-,F,-,1,2,...,F,T,T,T,T,T,F,0,0,1
32464,1700,"(25.0503178, 121.5272598, 25.0503178, 121.5272...",1,9000,2.0,-,F,-,0,2,...,F,T,T,T,T,T,F,0,0,1
32465,1710,"(25.0589044, 121.5650259, 25.0589044, 121.5650...",2,32000,2.0,T,F,-,2,0,...,F,F,T,F,F,F,T,0,0,1
32466,1706,"(25.0966454, 121.5401611, 25.0966454, 121.5401...",1,18000,1.0,F,T,T,1,1,...,T,T,T,T,T,T,T,0,0,1
32467,1709,"(24.9858373, 121.5671734, 24.9858373, 121.5671...",2,21000,2.0,T,F,-,2,0,...,F,F,T,F,F,F,T,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692516,705,"(25.129492, 121.72065, 25.129492, 121.72065)",0,12000,2.0,T,F,-,2,0,...,F,F,F,F,F,F,T,1,0,0
692517,705,"(25.1329302, 121.7288804, 25.1329302, 121.7288...",0,7500,2.0,F,F,-,1,0,...,F,T,T,F,F,F,F,1,0,0
692518,702,"(25.145617, 121.766862, 25.145617, 121.766862)",1,4000,1.0,-,F,-,1,2,...,F,T,T,F,T,T,F,1,0,0
692519,704,"(25.1286248, 121.7542348, 25.1286248, 121.7542...",1,6600,2.0,F,F,-,0,1,...,F,T,T,T,T,F,T,1,0,0


### 經緯度處理

1. 首先，刪除沒有登記經緯度的資料
2. 接著，利用字串處理，增加兩屬性，分別為經度和緯度。
3. 最後，再將'約略地點範圍'屬性刪除

In [22]:
df = df[df['約略地點範圍'] != '-']
place = df['約略地點範圍']
place = np.array(place)
lat = []
lon = []
for i in range(len(place)):
    x = place[i].split(',')
    lat.append(float(x[0].replace('(', '')))
    lon.append(float(x[1]))

df['緯度'] = lat
df['經度'] = lon
df = df.drop(['約略地點範圍'], axis=1)
df

C:\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


,鄉鎮市區,房屋曾出租過,月租金,押金月數,需要管理費？,提供車位？,需要停車費？,建築類型,物件類型,自報頂加？,...,提供家具_衣櫃？,提供家具_冰箱？,提供家具_網路？,提供家具_第四台？,提供家具_天然瓦斯？,代表_基隆市,代表_新北市,代表_臺北市,緯度,經度
32463,1707,1,10200,2.0,-,F,-,1,2,F,...,T,T,T,T,F,0,0,1,25.068092,121.519283
32464,1700,1,9000,2.0,-,F,-,0,2,F,...,T,T,T,T,F,0,0,1,25.050318,121.527260
32465,1710,2,32000,2.0,T,F,-,2,0,F,...,T,F,F,F,T,0,0,1,25.058904,121.565026
32466,1706,1,18000,1.0,F,T,T,1,1,F,...,T,T,T,T,T,0,0,1,25.096645,121.540161
32467,1709,2,21000,2.0,T,F,-,2,0,F,...,T,F,F,F,T,0,0,1,24.985837,121.567173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692516,705,0,12000,2.0,T,F,-,2,0,F,...,F,F,F,F,T,1,0,0,25.129492,121.720650
692517,705,0,7500,2.0,F,F,-,1,0,F,...,T,F,F,F,F,1,0,0,25.132930,121.728880
692518,702,1,4000,1.0,-,F,-,1,2,F,...,T,F,T,T,F,1,0,0,25.145617,121.766862
692519,704,1,6600,2.0,F,F,-,0,1,F,...,T,T,T,F,T,1,0,0,25.128625,121.754235


## 處理類型屬性

### 建築類型

1. 首先，讀取建物類型的編碼後，利用pandas的merge將兩表以編號為基準合併
2. 接著，刪除合併後多餘的屬性

In [23]:
con = pd.read_csv('tw-rental-data\編碼表\編碼表.建物類型.csv')
df = df.merge(con, left_on='建築類型', right_on='編碼')
dummies = pd.get_dummies( df.loc[:, '代表'], prefix='代表' ) 
df = pd.concat( [df, dummies], axis = 1 )
df = df.drop(['代表', '發行日期', '編碼', '建築類型'], axis=1)
df

,鄉鎮市區,房屋曾出租過,月租金,押金月數,需要管理費？,提供車位？,需要停車費？,物件類型,自報頂加？,所在樓層,...,提供家具_第四台？,提供家具_天然瓦斯？,代表_基隆市,代表_新北市,代表_臺北市,緯度,經度,代表_公寓,代表_透天,代表_電梯大樓
0,1707,1,10200,2.0,-,F,-,2,F,2,...,T,F,0,0,1,25.068092,121.519283,0,1,0
1,1706,1,18000,1.0,F,T,T,1,F,1,...,T,T,0,0,1,25.096645,121.540161,0,1,0
2,1706,1,25000,-,F,F,-,0,F,2,...,F,T,0,0,1,25.090508,121.524019,0,1,0
3,1702,1,12000,2.0,F,F,-,1,F,2,...,T,F,0,0,1,25.042881,121.573943,0,1,0
4,1706,1,13000,2.0,F,T,T,2,F,2,...,T,F,0,0,1,25.096406,121.524794,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285582,702,0,10000,2.0,F,F,-,0,F,12,...,F,F,1,0,0,25.137314,121.792435,0,0,1
285583,705,0,7000,2.0,T,F,-,1,F,9,...,F,T,1,0,0,25.138541,121.715154,0,0,1
285584,702,1,4800,2.0,-,T,T,2,F,14,...,T,F,1,0,0,25.137995,121.793910,0,0,1
285585,702,1,11000,2.0,F,F,-,0,F,8,...,T,F,1,0,0,25.138257,121.795362,0,0,1


### 物件類型

1. 首先，讀取物件類型的編碼後，利用pandas的merge將兩表以編號為基準合併
2. 接著，刪除合併後多餘的屬性

In [24]:
obj = pd.read_csv('tw-rental-data\編碼表\編碼表.物件類型.csv')
df = df.merge(obj, left_on='物件類型', right_on='編碼')
dummies = pd.get_dummies( df.loc[:, '代表'], prefix='代表' ) 
df = pd.concat( [df, dummies], axis = 1 )
df = df.drop(['代表', '發行日期', '編碼', '物件類型'], axis=1)
df

,鄉鎮市區,房屋曾出租過,月租金,押金月數,需要管理費？,提供車位？,需要停車費？,自報頂加？,所在樓層,坪數,...,代表_臺北市,緯度,經度,代表_公寓,代表_透天,代表_電梯大樓,代表_分租套房,代表_整層住家,代表_獨立套房,代表_雅房
0,1707,1,10200,2.0,-,F,-,F,2,5.0,...,1,25.068092,121.519283,0,1,0,1,0,0,0
1,1706,1,13000,2.0,F,T,T,F,2,10.0,...,1,25.096406,121.524794,0,1,0,1,0,0,0
2,1711,2,8000,2.0,-,F,-,F,1,6.6,...,1,25.042458,121.501625,0,1,0,1,0,0,0
3,1706,1,13000,2.0,F,T,T,F,2,10.0,...,1,25.096406,121.524794,0,1,0,1,0,0,0
4,1709,2,8500,2.0,-,F,-,F,1,5.0,...,1,24.990190,121.564288,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285582,704,0,3800,-,F,F,-,F,4,3.0,...,0,25.134069,121.764972,0,0,1,0,0,0,1
285583,701,1,3700,2.0,F,F,-,F,6,6.0,...,0,25.150909,121.728095,0,0,1,0,0,0,1
285584,702,0,3000,1.0,F,F,-,F,11,6.0,...,0,25.136243,121.786119,0,0,1,0,0,0,1
285585,702,0,3000,1.0,F,F,-,F,15,6.0,...,0,25.137319,121.794020,0,0,1,0,0,0,1


## 確定此交易以成立

### 針對此筆案件的狀態篩選出已完成之交易

1. 在資料集的說明中，'房屋曾出租過'此屬性若為2，便代表此交易以完成
2. 因此，篩選出資料集中'房屋曾出租過'的狀態為2的資料

In [25]:
df = df[df['房屋曾出租過'] == 2]
dfCity = df
df = df.drop(['鄉鎮市區'], axis=1)
df

,房屋曾出租過,月租金,押金月數,需要管理費？,提供車位？,需要停車費？,自報頂加？,所在樓層,坪數,陽台數,...,代表_臺北市,緯度,經度,代表_公寓,代表_透天,代表_電梯大樓,代表_分租套房,代表_整層住家,代表_獨立套房,代表_雅房
2,2,8000,2.0,-,F,-,F,1,6.6,-,...,1,25.042458,121.501625,0,1,0,1,0,0,0
4,2,8500,2.0,-,F,-,F,1,5.0,-,...,1,24.990190,121.564288,0,1,0,1,0,0,0
6,2,9500,2.0,-,F,-,F,2,10.0,0,...,1,25.050234,121.542199,0,1,0,1,0,0,0
22,2,11000,2.0,-,F,-,F,6,8.9,-,...,1,25.069666,121.511494,0,1,0,1,0,0,0
29,2,6000,1.0,-,F,-,F,3,4.6,-,...,1,24.986602,121.548142,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285562,2,6500,2.0,F,F,-,F,9,28.0,-,...,0,25.093698,121.755338,0,0,1,0,0,0,1
285564,2,6500,1.0,F,F,-,F,11,8.0,-,...,0,25.122426,121.720940,0,0,1,0,0,0,1
285570,2,12500,2.0,-,T,T,F,11,30.0,-,...,0,25.129175,121.737106,0,0,1,0,0,0,1
285574,2,3000,2.0,F,F,-,F,14,3.5,-,...,0,25.136435,121.786385,0,0,1,0,0,0,1


# 處理資料

## 整層住家

### 刪除不完整資料

1. 首先，選取整層住家後刪除不必要的屬性
2. 經過觀察發現，在資料集中，沒有填'附近有_...?'、'附近的...'和'提供家具_...?'和'...數'的都是同一批人，因此將不完整之資料刪除
3. 刪除'可炊'和'可寵'為'-'之缺失資料
4. 接著，利用'需要停車費？'和'提供車位？'這兩屬性利用虛擬變數的操作得出'提供車位且需要停車費'和'提供車位但不需要停車費'這兩項屬性，再刪除不必要之資料
5. 從表中可以得知，'附近的高鐵站數'和'有性別限制？'的值都是整行一樣，因此將屬性刪除(不影響)

In [26]:
df_whole = df[df['代表_整層住家'] == 1]
df_whole = df_whole.drop(['房屋曾出租過', '需要管理費？', '代表_分租套房', '代表_整層住家', '代表_獨立套房', '代表_雅房'], axis=1)
df_whole = df_whole[df_whole['陽台數'] != '-']
df_whole = df_whole[df_whole['附近有_學校？'] != '-']
df_whole = df_whole[df_whole['附近的公車站數'] != '-']
df_whole = df_whole[df_whole['提供家具_椅子？'] != '-']
df_whole = df_whole[df_whole['可炊？'] != '-']
df_whole = df_whole[df_whole['可寵？'] != '-']

In [27]:
df_whole['需要停車費？'] = df_whole['需要停車費？'].replace('-', '0')
df_whole['需要停車費？'] = df_whole['需要停車費？'].replace('F', '0')
df_whole['需要停車費？'] = df_whole['需要停車費？'].replace('T', '1')
df_whole['提供車位？'] = df_whole['提供車位？'].replace('-', '0')
df_whole['提供車位？'] = df_whole['提供車位？'].replace('F', '0')
df_whole['提供車位？'] = df_whole['提供車位？'].replace('T', '1')
dummies = pd.get_dummies( df_whole.loc[:, '需要停車費？'], prefix='需要停車費？') 
df_whole = pd.concat( [df_whole, dummies], axis = 1 )
df_whole['提供車位且需要停車費'] = df_whole['提供車位？'].astype(int)*df_whole['需要停車費？_1'].astype(int)
df_whole['提供車位但不需要停車費'] = df_whole['提供車位？'].astype(int)*df_whole['需要停車費？_0'].astype(int)
df_whole = df_whole.drop(['需要停車費？', '提供車位？', '需要停車費？_1', '需要停車費？_0'], axis=1)
for i in df_whole.columns:
    print(i)
    print(df_whole.loc[:, i].value_counts())

月租金
25000    1053
20000     980
18000     871
30000     794
15000     763
         ... 
43200       1
26820       1
26850       1
18700       1
2           1
Name: 月租金, Length: 874, dtype: int64
押金月數
2.0        24522
-           2358
1.0          192
3.0            9
1.42857        2
2.22222        1
1.66667        1
1.15385        1
1.50004        1
3.62069        1
2.14286        1
1.89873        1
2.85714        1
1.53846        1
3.0303         1
1.30911        1
2.2            1
2.1875         1
2.63158        1
1.72414        1
1.95652        1
2.16216        1
Name: 押金月數, dtype: int64
自報頂加？
F    26459
T      641
Name: 自報頂加？, dtype: int64
所在樓層
 4     3684
 3     3614
 5     3324
 2     3145
 6     2271
 7     1625
 8     1352
 9     1206
 1     1197
 10    1188
 11    1067
 12     916
 13     528
 14     448
 15     292
 18     181
 16     179
 0      153
 17     150
 19      93
 20      91
 21      74
 23      59
 22      55
 24      44
 25      24
-1       23
 27      22
 28   

In [28]:
df_whole = df_whole.drop(['附近的高鐵站數', '有性別限制？'], axis=1)

### 填補遺失值

1. 從資料可以得知，只有'押金月數'存在遺失值
2. 我們利用此項屬性的平均值來填補遺失值

In [29]:
month = np.array(df_whole['押金月數']).tolist()
while '-' in month:
    month.remove('-')
intMonth = [float(s) for s in month]
df_whole['押金月數'] = df_whole['押金月數'].replace('-', str(np.mean(intMonth)))

### 增加相關屬性

1. 在討論過後，我們認為對於公寓住戶來說，他們將更prefer低樓層，因此將兩屬性相乘

In [30]:
df_whole['公寓*所在樓層'] = df_whole['代表_公寓']*df_whole['所在樓層']
df_whole['電梯大樓*所在樓層'] = df_whole['代表_電梯大樓']*df_whole['所在樓層']

### 將T和F轉換成1和0

1. 為了方便做迴歸，先將F轉換成0並將T轉換成1
2. 將str轉換成int

In [31]:
df_whole = df_whole.replace('F', str(0))
df_whole = df_whole.replace('T', str(1))
df_whole[df_whole.columns] = df_whole[df_whole.columns].astype(float)
df_whole

,月租金,押金月數,自報頂加？,所在樓層,坪數,陽台數,衛浴數,房數,客廳數,額外費用_電費？,...,代表_臺北市,緯度,經度,代表_公寓,代表_透天,代表_電梯大樓,提供車位且需要停車費,提供車位但不需要停車費,公寓*所在樓層,電梯大樓*所在樓層
125761,19000.0,2.00000,0.0,0.0,42.00,2.0,2.0,3.0,2.0,1.0,...,1.0,24.989586,121.543020,0.0,1.0,0.0,0.0,0.0,0.0,0.0
125772,21000.0,1.99263,0.0,3.0,15.00,0.0,1.0,1.0,1.0,1.0,...,1.0,25.058368,121.526363,0.0,1.0,0.0,0.0,0.0,0.0,0.0
125810,250000.0,2.00000,0.0,1.0,122.11,4.0,5.0,5.0,2.0,1.0,...,1.0,25.039043,121.573211,0.0,1.0,0.0,0.0,1.0,0.0,0.0
125811,55000.0,2.00000,0.0,0.0,70.10,5.0,3.0,4.0,3.0,1.0,...,1.0,25.011186,121.568645,0.0,1.0,0.0,0.0,1.0,0.0,0.0
125812,19000.0,1.99263,0.0,2.0,27.00,1.0,1.0,1.0,1.0,1.0,...,1.0,25.064225,121.532130,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271080,11000.0,2.00000,0.0,7.0,13.00,1.0,1.0,2.0,1.0,1.0,...,0.0,25.139780,121.710325,0.0,0.0,1.0,0.0,0.0,0.0,7.0
271086,11000.0,2.00000,0.0,8.0,21.00,1.0,1.0,2.0,1.0,1.0,...,0.0,25.139286,121.795360,0.0,0.0,1.0,0.0,1.0,0.0,8.0
271098,16000.0,2.00000,0.0,2.0,42.00,2.0,2.0,4.0,2.0,1.0,...,0.0,25.140589,121.708876,0.0,0.0,1.0,0.0,0.0,0.0,2.0
271126,17000.0,2.00000,0.0,7.0,30.00,2.0,1.0,2.0,2.0,1.0,...,0.0,25.122833,121.741767,0.0,0.0,1.0,1.0,0.0,0.0,7.0


### 輸出檔案

In [32]:
df_whole.to_csv('2019_整層住家.csv', encoding='UTF-8', index=False)

# 建模

## 房屋屬性

In [41]:
testx = df[['月租金', '房數', '客廳數', '坪數', '代表_公寓', '代表_透天', '代表_電梯大樓', 
                    '代表_整層住家', '自報頂加？', '所在樓層', '陽台數', '衛浴數']]
testx = testx[testx['代表_整層住家']==1]
testx = testx[testx['房數']!='-']
testx = testx.replace('F', str(0))
testx = testx.replace('T', str(1))
testx = testx.astype(float)
testy = testx['月租金']
testx = testx.drop(['月租金', '代表_整層住家'], axis=1)

testx = sm.add_constant(testx)
est = sm.OLS(testy, testx)
est1 = est.fit()
print(est1.summary())

                            OLS Regression Results                            
Dep. Variable:                    月租金   R-squared:                       0.525
Model:                            OLS   Adj. R-squared:                  0.524
Method:                 Least Squares   F-statistic:                     6140.
Date:                Thu, 24 Dec 2020   Prob (F-statistic):               0.00
Time:                        18:27:23   Log-Likelihood:            -5.5686e+05
No. Observations:               50087   AIC:                         1.114e+06
Df Residuals:                   50077   BIC:                         1.114e+06
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       3835.7780    218.761     17.534      0.0

## 交通機能與環境屬性

In [42]:
testx = df[['月租金', '附近有_學校？', '附近有_公園？', '附近有_百貨公司？', '附近有_超商？', '附近有_傳統市場？', '附近有_夜市？', 
            '附近有_醫療機構？', '附近的捷運站數', '附近的公車站數', '附近的火車站數', '附近的公共自行車數（實驗中）', '代表_整層住家']]
testx = testx[testx['代表_整層住家']==1]
testx = testx[testx['附近有_學校？']!='-']
testx = testx[testx['附近的公車站數']!='-']
testx = testx.replace('F', str(0))
testx = testx.replace('T', str(1))
testx = testx.astype(float)
testy = testx['月租金']
testx = testx.drop(['月租金', '代表_整層住家'], axis=1)

testx = sm.add_constant(testx)
est = sm.OLS(testy, testx)
est1 = est.fit()
print(est1.summary())

                            OLS Regression Results                            
Dep. Variable:                    月租金   R-squared:                       0.070
Model:                            OLS   Adj. R-squared:                  0.069
Method:                 Least Squares   F-statistic:                     189.5
Date:                Thu, 24 Dec 2020   Prob (F-statistic):               0.00
Time:                        18:27:26   Log-Likelihood:            -3.1967e+05
No. Observations:               27908   AIC:                         6.394e+05
Df Residuals:                   27896   BIC:                         6.395e+05
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const           2.512e+04   1211.308     20.

## 總體

### Initial

In [33]:
df_whole.columns

Index(['月租金', '押金月數', '自報頂加？', '所在樓層', '坪數', '陽台數', '衛浴數', '房數', '客廳數',
       '額外費用_電費？', '額外費用_水費？', '額外費用_瓦斯？', '額外費用_網路？', '額外費用_第四台？', '附近有_學校？',
       '附近有_公園？', '附近有_百貨公司？', '附近有_超商？', '附近有_傳統市場？', '附近有_夜市？', '附近有_醫療機構？',
       '附近的捷運站數', '附近的公車站數', '附近的火車站數', '附近的公共自行車數（實驗中）', '有身份限制？', '可炊？',
       '可寵？', '提供家具_床？', '提供家具_桌子？', '提供家具_椅子？', '提供家具_電視？', '提供家具_熱水器？',
       '提供家具_冷氣？', '提供家具_沙發？', '提供家具_洗衣機？', '提供家具_衣櫃？', '提供家具_冰箱？', '提供家具_網路？',
       '提供家具_第四台？', '提供家具_天然瓦斯？', '代表_基隆市', '代表_新北市', '代表_臺北市', '緯度', '經度',
       '代表_公寓', '代表_透天', '代表_電梯大樓', '提供車位且需要停車費', '提供車位但不需要停車費', '公寓*所在樓層',
       '電梯大樓*所在樓層'],
      dtype='object')

In [43]:
y = df_whole['月租金']
x = df_whole.drop(['月租金', '經度', '緯度', '額外費用_電費？', '額外費用_水費？', '額外費用_瓦斯？', '額外費用_網路？', '額外費用_第四台？', '提供家具_床？', '提供家具_桌子？', '提供家具_椅子？', '提供家具_電視？', '提供家具_熱水器？',
       '提供家具_冷氣？', '提供家具_沙發？', '提供家具_洗衣機？', '提供家具_衣櫃？', '提供家具_冰箱？', '提供家具_網路？',
       '提供家具_第四台？', '提供家具_天然瓦斯？', '所在樓層'], axis=1)

x = sm.add_constant(x)
est = sm.OLS(y, x)
est1 = est.fit()
print(est1.summary())

                            OLS Regression Results                            
Dep. Variable:                    月租金   R-squared:                       0.680
Model:                            OLS   Adj. R-squared:                  0.680
Method:                 Least Squares   F-statistic:                     1982.
Date:                Thu, 24 Dec 2020   Prob (F-statistic):               0.00
Time:                        18:29:08   Log-Likelihood:            -2.9547e+05
No. Observations:               27100   AIC:                         5.910e+05
Df Residuals:                   27070   BIC:                         5.912e+05
Df Model:                          29                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const          -3466.4592   1273.555     -2.

### delete attribute that p > 0.5

In [44]:
x = df_whole.drop(['月租金', '經度', '緯度', '額外費用_電費？', '額外費用_水費？', '額外費用_瓦斯？', '額外費用_網路？', '額外費用_第四台？', '提供家具_床？', '提供家具_桌子？', '提供家具_椅子？', '提供家具_電視？', '提供家具_熱水器？',
       '提供家具_冷氣？', '提供家具_沙發？', '提供家具_洗衣機？', '提供家具_衣櫃？', '提供家具_冰箱？', '提供家具_網路？',
       '提供家具_第四台？', '提供家具_天然瓦斯？', '所在樓層', '可炊？', '附近有_夜市？'], axis=1)

x = sm.add_constant(x)
est = sm.OLS(y, x)
est1 = est.fit()
print(est1.summary())

                            OLS Regression Results                            
Dep. Variable:                    月租金   R-squared:                       0.680
Model:                            OLS   Adj. R-squared:                  0.680
Method:                 Least Squares   F-statistic:                     2129.
Date:                Thu, 24 Dec 2020   Prob (F-statistic):               0.00
Time:                        18:30:25   Log-Likelihood:            -2.9547e+05
No. Observations:               27100   AIC:                         5.910e+05
Df Residuals:                   27072   BIC:                         5.912e+05
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const          -3399.8632   1242.911     -2.

# 繪圖資料轉換

## 各房型、建築類型在三區中分別的月租金

1. 首先，讀取資料集後刪除不必要的屬性
2. 接著，讀取縣市的資料集，利用merge合併之後篩選出地區符合北北基的案件
3. 再來，將案件訂為已成交之案件
4. 接著，分別讀取各編碼資料集，利用merge的方式將房型、建築類型和區加在各案件的後面做為新的屬性
5. 最後，只保留"月租金", "縣市", "物件類型", "建築類型", "鄉鎮市區"這五項屬性並輸出成csv檔。

In [54]:
df1 = pd.read_csv('tw-rental-data/2019-dedup.csv')
df1 = df1.drop(['重複物件數', '最大物件編號', '最小物件編號', '最大物件首次發現時間', '最小物件首次發現時間', '租屋平台', 
              '最後出租時間', '最大出租所費天數', '押金類型', '押金金額', '月管理費', '月停車費', '每坪租金（含管理費與停車費）', 
              '建物樓高', '距頂樓層數', '格局編碼（陽台/衛浴/房/廳）', '性別限制', '有產權登記？', '刊登者類型', '最大刊登者編碼', 
              '仲介資訊'], axis=1)
city1 = pd.read_csv('tw-rental-data/編碼表/編碼表.縣市.csv')
df1 = df1.merge(city1, left_on='縣市', right_on='編碼')
df1 = df1.drop(['縣市', '編碼', '發行日期'], axis=1)
mask1 = df1['代表'] == '臺北市'
mask2 = df1['代表'] == '新北市'
mask3 = df1['代表'] == '基隆市'
df1 = df1[(mask1 | mask2 | mask3)]
df1 = df1[df1['約略地點範圍'] != '-']
df1 = df1[df1['房屋曾出租過'] == 2]
df1 = df1.rename(columns={'代表':'縣市'})
obj1 = pd.read_csv('tw-rental-data\編碼表\編碼表.物件類型.csv')
df1 = df1.merge(obj1, left_on='物件類型', right_on='編碼')
df1 = df1.drop(['發行日期', '編碼', '物件類型'], axis=1)
df1 = df1.rename(columns={'代表':'物件類型'})
con1 = pd.read_csv('tw-rental-data\編碼表\編碼表.建物類型.csv')
df1 = df1.merge(con1, left_on='建築類型', right_on='編碼')
df1 = df1.drop(['發行日期', '編碼', '建築類型'], axis=1)
df1 = df1.rename(columns={'代表':'建築類型'})
city1 = pd.read_csv('tw-rental-data\編碼表\編碼表.鄉鎮市區.csv')
df1 = df1.merge(city1, left_on='鄉鎮市區', right_on='編碼')
df1 = df1.drop(['發行日期', '編碼', '鄉鎮市區'], axis=1)
df1 = df1.rename(columns={'代表':'鄉鎮市區'})
df1

C:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (12,13,27) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,約略地點範圍,房屋曾出租過,月租金,押金月數,需要管理費？,提供車位？,需要停車費？,自報頂加？,所在樓層,坪數,...,提供家具_洗衣機？,提供家具_衣櫃？,提供家具_冰箱？,提供家具_網路？,提供家具_第四台？,提供家具_天然瓦斯？,縣市,物件類型,建築類型,鄉鎮市區
0,"(25.0589044, 121.5650259, 25.0589044, 121.5650...",2,32000,2.0,T,F,-,F,4,41.00,...,F,T,F,F,F,T,臺北市,整層住家,電梯大樓,臺北市松山區
1,"(25.0498657, 121.5707321, 25.0498657, 121.5707...",2,28000,2.0,T,F,-,F,13,25.05,...,T,T,T,F,F,T,臺北市,整層住家,電梯大樓,臺北市松山區
2,"(25.0514686, 121.5781832, 25.0514686, 121.5781...",2,38000,2.0,T,F,-,F,7,28.00,...,T,T,T,T,T,T,臺北市,整層住家,電梯大樓,臺北市松山區
3,"(25.0554224, 121.5538564, 25.0554224, 121.5538...",2,53000,2.0,T,T,T,F,6,48.00,...,T,T,T,F,F,T,臺北市,整層住家,電梯大樓,臺北市松山區
4,"(25.051356, 121.5603984, 25.051356, 121.5603984)",2,16000,2.0,T,F,-,T,8,15.00,...,T,T,T,F,F,F,臺北市,整層住家,電梯大樓,臺北市松山區
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92679,"(25.2777445, 121.5206175, 25.2777445, 121.5206...",2,8000,2.0,F,T,F,F,1,30.00,...,F,F,F,F,F,T,新北市,整層住家,透天,新北市石門區
92680,"(25.2891016, 121.5467379, 25.2891016, 121.5467...",2,15000,2.0,F,F,-,F,0,50.00,...,-,-,-,-,-,-,新北市,整層住家,透天,新北市石門區
92681,"(25.0390449, 121.8650526, 25.0390449, 121.8650...",2,5000,2.0,-,F,-,F,2,7.00,...,T,T,T,F,T,F,新北市,分租套房,公寓,新北市雙溪區
92682,"(24.981951, 121.658433, 24.981951, 121.658433)",2,30000,2.0,F,F,-,F,0,150.00,...,F,F,F,F,F,F,新北市,整層住家,透天,新北市石碇區


In [55]:
df1 = df1[["月租金", "縣市", "物件類型", "建築類型", "鄉鎮市區"]]
df1

,月租金,縣市,物件類型,建築類型,鄉鎮市區
0,32000,臺北市,整層住家,電梯大樓,臺北市松山區
1,28000,臺北市,整層住家,電梯大樓,臺北市松山區
2,38000,臺北市,整層住家,電梯大樓,臺北市松山區
3,53000,臺北市,整層住家,電梯大樓,臺北市松山區
4,16000,臺北市,整層住家,電梯大樓,臺北市松山區
...,...,...,...,...,...
92679,8000,新北市,整層住家,透天,新北市石門區
92680,15000,新北市,整層住家,透天,新北市石門區
92681,5000,新北市,分租套房,公寓,新北市雙溪區
92682,30000,新北市,整層住家,透天,新北市石碇區


In [64]:
df1.to_csv('總表.csv', index=True)

## 經緯度對應月租金

1. 在最上面的資料處理中，我們已經將經緯度分別切割好了，因此，只要將"月租金", "經度", "緯度"篩選出來建成dataframe
2. 將此資料表輸出成csv

In [62]:
df2 = df[["月租金", "經度", "緯度"]]
df2 = df2.reset_index()
df2 = df2.drop(['index'], axis=1)
df2

,月租金,經度,緯度
0,8000,121.501625,25.042458
1,8500,121.564288,24.990190
2,9500,121.542199,25.050234
3,11000,121.511494,25.069666
4,6000,121.548142,24.986602
...,...,...,...
92679,6500,121.755338,25.093698
92680,6500,121.720940,25.122426
92681,12500,121.737106,25.129175
92682,3000,121.786385,25.136435


In [66]:
df2.to_csv('經緯度.csv', index=True)